## Sommaire: <a class="anchor" id="sommaire"></a>
* [Sommaire](#sommaire)
* [Preambule](#prem)
     * [Package Loading](#package)
     * [Functions](#function)
* [GRU](#lstm)
    * [1.FD001](#fd001)
        * [1.1 Data loading](#fd001dataload)
        * [1.2 Model selection](#fd001modelselect)
    * [2.FD002](#fd002)
         * [2.1 Data loading](#fd002dataload)
         * [2.2 Model selection](#fd002modelselect)
    * [3.FD003](#fd003)
         * [3.1 Data loading](#fd003dataload)
         * [3.2 Model selection](#fd003modelselect)
    * [4.FD004](#fd004)
         * [4.1 Data loading](#fd004dataload)
         * [4.2 Model selection](#fd004modelselect)

### Package loading <a class = "anchor" id="package"></a>

In [1]:
from __future__ import print_function
import pandas as pd
import matplotlib.pyplot as plt

from lime.lime_tabular import RecurrentTabularExplainer
from tqdm import tqdm
import keras
from sp_modif.model_function import *
from sp_modif.methods import *
from sp_modif.data_prep import *
from sp_modif.evaluator import *
from sp_modif.SHAP import *
from sp_modif.L2X import *
import warnings

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import random
# import keras
import math

from sklearn.metrics import mean_squared_error, r2_score 
from sklearn.model_selection import GroupKFold
from sklearn import preprocessing
from keras import backend as K
from sklearn.preprocessing import MinMaxScaler , StandardScaler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Activation, GRU
from scipy import optimize
from methods import *
import warnings
from tensorflow.keras import optimizers
warnings.filterwarnings("ignore")
%matplotlib inline

warnings.filterwarnings('ignore')
print("okay")

okay


### Functions <a class = "anchor" id="function"></a>

In [2]:
# Fixer le seed pour la reproductibilité
SEED = 0
def set_seed(seed=SEED):
    os.environ['PYTHONHASHSEED'] = str(SEED)
    random.seed(SEED)
    np.random.seed(SEED)
    tf.random.set_seed(SEED)

# Appeler la fonction pour fixer le seed
set_seed(SEED)

In [9]:
def rul_piecewise_fct(X_train, rul):
    
    X_train['RUL'].clip(upper=rul, inplace=True)
    
    return X_train

def prep_data(train, test, drop_sensors, remaining_sensors, alpha):
    X_train_interim = add_operating_condition(train.drop(drop_sensors, axis=1))
    X_test_interim = add_operating_condition(test.drop(drop_sensors, axis=1))

    X_train_interim, X_test_interim = condition_scaler(X_train_interim, X_test_interim, remaining_sensors)

    X_train_interim = exponential_smoothing(X_train_interim, remaining_sensors, 0, alpha)
    X_test_interim = exponential_smoothing(X_test_interim, remaining_sensors, 0, alpha)
    
    return X_train_interim, X_test_interim

# Models functions

# 1layers
def model_gru_1layer(input_shape, nodes_per_layer, dropout, activation, weights_file):
    
    cb = keras.callbacks.EarlyStopping(monitor='loss', patience=4)
    model = Sequential()
    model.add(GRU(units = nodes_per_layer, activation=activation, 
                  input_shape=input_shape))
    model.add(Dropout(dropout))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',
                  optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001))
    model.save_weights(weights_file)

    return model
# 2layers
def model_gru_2layer(input_shape, nodes_per_layer, dropout, activation, weights_file):
    
    cb = keras.callbacks.EarlyStopping(monitor='loss', patience=4)
    model = Sequential()
    model.add(GRU(units = nodes_per_layer, activation=activation, 
                  input_shape=input_shape, return_sequences=True))
    model.add(Dropout(dropout))
    model.add(GRU(units = nodes_per_layer, activation=activation, 
                  input_shape=input_shape))
    model.add(Dropout(dropout))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',
                  optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001))
    model.save_weights(weights_file)

    return model

# 3layers
def model_gru_3layer(input_shape, nodes_per_layer, dropout, activation, weights_file):
    
    cb = keras.callbacks.EarlyStopping(monitor='loss', patience=4)
    model = Sequential()
    model.add(GRU(units = nodes_per_layer, activation=activation, 
                  input_shape=input_shape, return_sequences=True))
    model.add(Dropout(dropout))
    model.add(GRU(units = nodes_per_layer, activation=activation, 
                  input_shape=input_shape, return_sequences=True))
    model.add(Dropout(dropout))
    model.add(GRU(units = nodes_per_layer, activation=activation, 
                  input_shape=input_shape))
    model.add(Dropout(dropout))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',
                  optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001))
    model.save_weights(weights_file)

    return model

def model_gru_4layer(input_shape, nodes_per_layer, dropout, activation, weights_file):
    
    cb = keras.callbacks.EarlyStopping(monitor='loss', patience=4)
    model = Sequential()
    model.add(GRU(units = nodes_per_layer, activation=activation, 
                  input_shape=input_shape, return_sequences=True))
    model.add(Dropout(dropout))
    model.add(GRU(units = nodes_per_layer, activation=activation, 
                  input_shape=input_shape, return_sequences=True))
    model.add(Dropout(dropout))
    model.add(GRU(units = nodes_per_layer, activation=activation, 
                  input_shape=input_shape, return_sequences=True))
    model.add(Dropout(dropout))
    model.add(GRU(units = nodes_per_layer, activation=activation, 
                  input_shape=input_shape))
    model.add(Dropout(dropout))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',
                  optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001))
    model.save_weights(weights_file)

    return model

# GRU

### 1. FD003 <a class = "anchor" id="fd031"></a>

In [4]:
# Data preparation
# Data loading

train, test, y_test = prepare_data('FD003.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']

remaining_sensors = ['T24','T30','T50','P30','Nf','Nc','Ps30','phi',
                'NRf','NRc','BPR','htBleed','W31','W32'] # selection based on main_notebook

drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

(24720, 27) (16596, 26) (100, 1)


In [5]:
rul_piecewise = 130
train['RUL'].clip(upper=rul_piecewise, inplace=True)

In [6]:
# Lower alpha's perform better, so we can ditch a few high ones to reduce the search space
alpha_list = [0.01, 0.05] + list(np.arange(10,60+1,10)/100)

sequence_list = list(np.arange(10,40+1,5))
epoch_list = list(np.arange(5,20+1,5))
nodes_list = [[32], [64], [128], [256]]

# lowest dropout=0.1, because I know zero dropout will yield better training results but worse generalization
dropouts = list(np.arange(1,4)/10)  

# again, earlier testing revealed relu performed significantly worse, so I removed it from the options
activation_functions = ['tanh']
batch_size_list = [32, 64, 128, 256]
sensor_list = [sensor_names]

tuning_options = np.prod([len(alpha_list),
                          len(sequence_list),
                          len(epoch_list),
                          len(nodes_list),
                          len(dropouts),
                          len(activation_functions),
                          len(batch_size_list),
                          len(sensor_list)])
tuning_options

10752

In [7]:
ITERATIONS = 25
SEED = 0

In [10]:
%%time
results = pd.DataFrame(columns=['RMSE', 'std_RMSE', 
                                'S_score','std_S_score',
                                'MSE', 'std_MSE',
                                'nodes', 'dropout',
                                'activation', 'batch_size'])


for i in range(ITERATIONS):
    if ITERATIONS < 10:
        print('iteration ', i+1)
    elif ((i+1) % 10 == 0):
        print('iteration ', i+1)    
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    
    # parameter's sample
    weights_file = "model_weight/weights_file_gru.h5"
    alpha = 0.3
    sequence_length = 30
    epochs = 25
    nodes_per_layer = random.sample(nodes_list, 1)[0]
    dropout = random.sample(dropouts, 1)[0]
    activation = random.sample(activation_functions, 1)[0]
    batch_size = random.sample(batch_size_list, 1)[0]
    remaining_sensors = remaining_sensors
    drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = model_gru_1layer(input_shape, nodes_per_layer[0], dropout,
                             activation, weights_file)
    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)
    
    # Model fitting
    with tf.device('/device:GPU:0'):
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=1)
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results = pd.concat([results, pd.DataFrame(d, index=[0])], ignore_index=True)

(21820, 30, 14) (21820, 1) (100, 30, 14)
Epoch 1/25
341/341 [==============================] - 8s 19ms/step - loss: 5413.0015 - val_loss: 2089.2949
Epoch 2/25
341/341 [==============================] - 6s 19ms/step - loss: 2580.4509 - val_loss: 1607.8918
Epoch 3/25
341/341 [==============================] - 6s 19ms/step - loss: 1978.9729 - val_loss: 1757.5923
Epoch 4/25
341/341 [==============================] - 6s 18ms/step - loss: 1894.9200 - val_loss: 1842.9081
Epoch 5/25
341/341 [==============================] - 7s 19ms/step - loss: 1336.4696 - val_loss: 595.5422
Epoch 6/25
341/341 [==============================] - 6s 19ms/step - loss: 501.4252 - val_loss: 540.9349
Epoch 7/25
341/341 [==============================] - 6s 19ms/step - loss: 333.1841 - val_loss: 330.1049
Epoch 8/25
341/341 [==============================] - 6s 19ms/step - loss: 287.5868 - val_loss: 340.4538
Epoch 9/25
341/341 [==============================] - 7s 19ms/step - loss: 269.4568 - val_loss: 324.0808
Epoch

In [11]:
results.sort_values(by='S_score').head(5)

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size
12,14.059276,0.0,338.142049,0.0,197.663223,0.0,[64],0.1,tanh,32
13,14.669097,0.0,375.602666,0.0,215.182419,0.0,[256],0.3,tanh,64
4,14.343987,0.0,422.703087,0.0,205.749908,0.0,[128],0.1,tanh,32
23,14.494882,0.0,453.122978,0.0,210.101654,0.0,[128],0.1,tanh,32
21,15.680173,0.0,542.117846,0.0,245.867767,0.0,[32],0.1,tanh,32


In [12]:
results.to_csv("fd003_result/optim_result_1layer.csv")

### FD004

In [13]:
# Data preparation
# Data loading

train, test, y_test = prepare_data('FD004.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']

remaining_sensors = ['T24','T30','T50','P30','Nf','Nc','Ps30','phi',
                'NRf','NRc','BPR','htBleed','W31','W32'] # selection based on main_notebook

drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

(61249, 27) (41214, 26) (248, 1)


In [14]:
rul_piecewise = 130
train['RUL'].clip(upper=rul_piecewise, inplace=True)

In [15]:
%%time
results = pd.DataFrame(columns=['RMSE', 'std_RMSE', 
                                'S_score','std_S_score',
                                'MSE', 'std_MSE',
                                'nodes', 'dropout',
                                'activation', 'batch_size'])


for i in range(ITERATIONS):
    if ITERATIONS < 10:
        print('iteration ', i+1)
    elif ((i+1) % 10 == 0):
        print('iteration ', i+1)    
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    
    # parameter's sample
    weights_file = "model_weight/weights_file_gru.h5"
    alpha = 0.3
    sequence_length = 30
    epochs = 25
    nodes_per_layer = random.sample(nodes_list, 1)[0]
    dropout = random.sample(dropouts, 1)[0]
    activation = random.sample(activation_functions, 1)[0]
    batch_size = random.sample(batch_size_list, 1)[0]
    remaining_sensors = remaining_sensors
    drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = model_gru_1layer(input_shape, nodes_per_layer[0], dropout,
                             activation, weights_file)
    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)
    
    # Model fitting
    with tf.device('/device:GPU:0'):
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=1)
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results = pd.concat([results, pd.DataFrame(d, index=[0])], ignore_index=True)

(54028, 30, 14) (54028, 1) (248, 30, 14)
Epoch 1/25
423/423 [==============================] - 23s 51ms/step - loss: 3340.7241 - val_loss: 2078.6707
Epoch 2/25
423/423 [==============================] - 22s 52ms/step - loss: 1249.3040 - val_loss: 1616.1948
Epoch 3/25
423/423 [==============================] - 22s 51ms/step - loss: 636.8220 - val_loss: 1842.1812
Epoch 4/25
423/423 [==============================] - 22s 52ms/step - loss: 523.3027 - val_loss: 2083.7278
Epoch 5/25
423/423 [==============================] - 23s 54ms/step - loss: 443.0068 - val_loss: 1945.6218
Epoch 6/25
423/423 [==============================] - 23s 53ms/step - loss: 389.4566 - val_loss: 1838.3813
Epoch 7/25
423/423 [==============================] - 22s 52ms/step - loss: 355.4983 - val_loss: 1785.2485
Epoch 8/25
423/423 [==============================] - 22s 51ms/step - loss: 351.1451 - val_loss: 1791.8473
Epoch 9/25
423/423 [==============================] - 22s 52ms/step - loss: 338.0566 - val_loss: 1716

In [16]:
results.sort_values(by='S_score').head(5)

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size
11,14.996016,0.0,1068.145958,0.0,224.880478,0.0,[256],0.3,tanh,64
13,15.494028,0.0,1149.516955,0.0,240.064926,0.0,[128],0.2,tanh,32
2,15.779557,0.0,1208.643000,0.0,248.994431,0.0,[64],0.2,tanh,32
23,15.471055,0.0,1210.752320,0.0,239.353577,0.0,[64],0.2,tanh,32
14,15.749801,0.0,1215.751911,0.0,248.056229,0.0,[256],0.1,tanh,64


In [17]:
results.to_csv("fd004_result/optim_result_1layer.csv")